<a href="https://colab.research.google.com/github/royam0820/crewai/blob/main/L6_collaboration_financial_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L6: Multi-agent Collaboration for Financial Analysis

This Jupyter notebook demonstrates a **financial analysis workflow using a multi-agent system doing agentic collaboration**. It leverages the crewai library to create and manage AI agents responsible for various tasks, including:
- data analysis,
- strategy development,
- execution planning, and
- risk assessment.

The workflow aims to provide a comprehensive analysis and trading strategy for selected stocks.

In this lesson, you will learn ways for making agents collaborate with each other.

## Library Installation

In [ ]:
!pip install crewai==0.28.8
!pip install crewai_tools==0.1.6
!pip install langchain_community==0.0.29 # this is formatted as code
!pip install utils

## Warnings Suppression

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Importing Required Classes

In [ ]:
from crewai import Agent, Task, Crew
import os
from google.colab import userdata

## Setting API Keys And LLM

In [ ]:
openai_api_key = userdata.get("OPENAI_API_KEY")
serper_api_key = userdata.get("SERPER_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["SERPER_API_KEY"] = serper_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

## Defining Tools

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Defining Agents

### Data Analyst

The Data Analyst agent is responsible for analyzing market data to identify trends.

In [ ]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

### Trading Strategy Developer

The Trading Strategy Developer agent develops and tests trading strategies.

In [ ]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

### Execution Planner

The Execution Planner agent formulates execution plans for trading strategies.

In [ ]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

### Risk Management Specialist

The Risk Management Specialist agent assesses and mitigates risks.

In [ ]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Defining Tasks

### Data Analysis Task

This task involves analyzing market data.

In [ ]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

### Strategy Development Task

This task involves developing and testing trading strategies.

In [ ]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


### Execution Planning Task

This task involves planning the execution of trading strategies.

In [ ]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


### Risk Assessment Task

This task involves assessing and mitigating risks.

In [ ]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Assembling the Crew and Executing the Workflow

- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [ ]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            risk_management_agent],

    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           risk_assessment_task],

    manager_llm=ChatOpenAI(model="gpt-3.5-turbo",
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

NB: The `Process` class in CrewAI defines how tasks are executed and coordinated among agents within a crew. It determines the workflow and orchestration strategy for the agents to collaborate and complete their assigned tasks.
- `Process.Hierarchical`: The process to be used for coordinating the agents and tasks. In this case, it's set to Process.hierarchical, which means that the tasks will be executed in a hierarchical manner.

`manager_llm=ChatOpenAI()`: The LLM that will act as the manager for the crew. In this case, it's an instance of ChatOpenAI with the gpt-3.5-turbo model and a temperature of 0.7 (which controls the randomness of the generated text). This manager agent will be automatically created for you by Crewai.

## Process types
CrewAI currently supports two main process types:
- **Sequential Process** (`Process.sequential`)
Tasks are executed one after the other in a predefined linear order. Ideal for projects with a clear step-by-step sequence of tasks.
Ensures orderly progression and easy monitoring of task completion.
- **Hierarchical Process** (`Process.hierarchical`)
Tasks are organized in a managerial hierarchy, mimicking a corporate structure.

A "`manager`" agent is automatically created to oversee task planning, delegation, and validation.
Tasks are not pre-assigned; the manager dynamically allocates tasks to agents as needed.

## Inputs and Execution

- Set the inputs for the execution of the crew.

In [ ]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst in order to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: 
{
"coworker": "Data Analyst",
"task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning",
"context": "The Data Analyst is responsible for continuously monitoring and analyzing market data for the selected stock (AAPL) using statistical modeling and machine learning techniques. The goal is to identify trends and predict market movements for AAPL."
}


> Entering new CrewAgentExecutor chain...
Thought: To effectively monitor and analyze market data 

### Display the final result as Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result)

**Risk Analysis Report for Day Trading Strategies on AAPL**

Based on the insights from the financial website data and considering the context shared for day trading AAPL stock, the risk analysis report highlights several key risk factors and mitigation strategies for successful day trading on Apple Inc. (AAPL).

**1. Market Volatility**
AAPL has shown recent price volatility, with a day's range between $190.63 and $192.10. This indicates a potential for significant intraday price movements which traders must be cautious of.

**2. Volume and Liquidity**
The average volume for AAPL is 62,739,854 shares, with today's volume at 12,960,713 shares. High liquidity implies that entry and exit from positions can be executed more smoothly, but the high volume also means that significant market movements can occur quickly.

**3. Earnings Announcements**
The next earnings date is scheduled between August 01, 2024, and August 05, 2024. Trading around earnings announcements can be risky due to potential drastic movements based on results versus expectations.

**4. Macro-economic Factors**
The overall market sentiment and macroeconomic factors can also impact AAPL's stock price. Movements in the S&P 500, Dow 30, and Nasdaq indices (currently experiencing downward trends) can influence investor sentiment and AAPL's stock performance.

**5. Competitive and Sector Risks**
AAPL faces competitive pressures, such as those from Nvidia in technological advancements. These factors can affect investor confidence and stock performance.

**Mitigation Strategies:**
- Setting Stop-Loss Orders: To manage risk effectively, setting stop-loss orders can help limit potential losses when unexpected movements occur.
- Diversification: Avoid overexposure to AAPL by diversifying the trading portfolio across different sectors and assets.
- Stay Informed: Constantly monitor market news and AAPL-specific developments. Adjust trading strategies accordingly, especially close to earnings announcements or major corporate events.
- Risk Capital Allocation: Allocate only a portion of risk capital to day trading AAPL to mitigate potential financial impacts.

These strategies should align with the firm's overall risk tolerance and trading goals, ensuring that trading activities are conducted within acceptable risk parameters.

# Conclusion

This notebook demonstrates how to automate financial analysis and trading strategy development using a multi-agent system, ensuring a comprehensive approach to market analysis, strategy development, execution planning, and risk management.
